# Figure 1C - YARN Normalization Version

Heatplot representing similarity in the fold-changes between male and female samples, with the values in the heatmap being the correlation between the vectors of fold changes of the tissues</b>

We downloaded the GTEx version 8.0 RNA-seq and genotype data (phs000424.v8.v2), released 2019-08-26.
We used YARN (https://bioconductor.org/packages/release/bioc/html/yarn.html), uploading the downloadGTEx function
to download this release, and used it to perform quality control, gene filtering and normalization pre-processing on the
GTEx RNA-seq data, as described in (Paulson et al, 2017).   This pipelines tested for sample sex-misidentification, 
merged related sub-tissues, performed tissue-aware normalization using qsmooth (Hicks et al, 2017).

In [1]:
if (!requireNamespace("BiocManager", quietly=TRUE))
    install.packages("BiocManager")
#BiocManager::install("yarn")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [2]:
BiocManager::install("downloader")

Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'BiocVersion', 'downloader'
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'backports', 'BH', 'bit', 'blob', 'broom', 'callr', 'caret',
  'cli', 'curl', 'data.table', 'DBI', 'devtools', 'digest', 'dplyr',
  'ellipsis', 'fansi', 'foreach', 'forecast', 'fracdiff', 'ggplot2', 'gh',
  'haven', 'hexbin', 'hms', 'htmltools', 'htmlwidgets', 'httpuv', 'IRkernel',
  'jsonlite', 'KernSmooth', 'knitr', 'later', 'lava', 'MASS', 'Matrix', 'mgcv',
  'mime', 'ModelMetrics', 'nlme', 'nycflights13', 'pillar', 'pkgbuild',
  'pkgconfig', 'plyr', 'prettyunits', 'processx', 'prodlim', 'promises', 'ps',
  'purrr', 'quadprog', 'R6', 'Rcpp', 'RcppArmadillo', 'RCurl', 'recipes',
  'repr', 'rlang', 'rmarkdown', 'roxygen2', 'RSQLite', 'rstudioapi', 'rvest',
  'scales', 'selectr', 'shiny', 'sparklyr', 'SQUAREM', 'stringi', 'survival',
  'sys', 'testthat', 'tidyr', 'tidyselect', 

In [3]:
BiocManager::install("readr")

Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'readr'
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'backports', 'BH', 'bit', 'blob', 'broom', 'callr', 'caret',
  'cli', 'curl', 'data.table', 'DBI', 'devtools', 'digest', 'dplyr',
  'ellipsis', 'fansi', 'foreach', 'forecast', 'fracdiff', 'ggplot2', 'gh',
  'haven', 'hexbin', 'hms', 'htmltools', 'htmlwidgets', 'httpuv', 'IRkernel',
  'jsonlite', 'KernSmooth', 'knitr', 'later', 'lava', 'MASS', 'Matrix', 'mgcv',
  'mime', 'ModelMetrics', 'nlme', 'nycflights13', 'pillar', 'pkgbuild',
  'pkgconfig', 'plyr', 'prettyunits', 'processx', 'prodlim', 'promises', 'ps',
  'purrr', 'quadprog', 'R6', 'Rcpp', 'RcppArmadillo', 'RCurl', 'recipes',
  'repr', 'rlang', 'rmarkdown', 'roxygen2', 'RSQLite', 'rstudioapi', 'rvest',
  'scales', 'selectr', 'shiny', 'sparklyr', 'SQUAREM', 'stringi', 'survival',
  'sys', 'testthat', 'tidyr', 'tidyselect', 'tidyverse', 'tinyte

In [22]:
BiocManager::install("biomaRt")

Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'biomaRt'
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'backports', 'BH', 'bit', 'blob', 'broom', 'callr', 'caret',
  'cli', 'curl', 'data.table', 'DBI', 'devtools', 'digest', 'dplyr',
  'ellipsis', 'fansi', 'foreach', 'forecast', 'fracdiff', 'ggplot2', 'gh',
  'haven', 'hexbin', 'hms', 'htmltools', 'htmlwidgets', 'httpuv', 'IRkernel',
  'jsonlite', 'KernSmooth', 'knitr', 'later', 'lava', 'MASS', 'Matrix', 'mgcv',
  'mime', 'ModelMetrics', 'nlme', 'nycflights13', 'pillar', 'pkgbuild',
  'pkgconfig', 'plyr', 'prettyunits', 'processx', 'prodlim', 'promises', 'ps',
  'purrr', 'quadprog', 'R6', 'Rcpp', 'RcppArmadillo', 'RCurl', 'recipes',
  'repr', 'rlang', 'rmarkdown', 'roxygen2', 'RSQLite', 'rstudioapi', 'rvest',
  'scales', 'selectr', 'shiny', 'sparklyr', 'SQUAREM', 'stringi', 'survival',
  'sys', 'testthat', 'tidyr', 'tidyselect', 'tidyverse', 'tiny

In [5]:
install.packages("devtools")

also installing the dependencies ‘rex’, ‘crosstalk’, ‘covr’, ‘DT’, ‘ellipsis’, ‘rversions’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [6]:
getwd()

[1] "/mnt/shared/ec2-user/session_data/lifebitCloudOSDRE/jupyter"

In [7]:
setwd("../..")
getwd()

[1] "/mnt/shared/ec2-user/session_data"

In [13]:
#library("downloader")
#library(readr)
BiocManager::install("biomaRt")
library("biomaRt")

Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'biomaRt'
also installing the dependencies ‘BiocGenerics’, ‘Biobase’, ‘IRanges’, ‘S4Vectors’, ‘XML’, ‘AnnotationDbi’, ‘BiocFileCache’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'backports', 'BH', 'bit', 'blob', 'broom', 'callr', 'caret',
  'cli', 'curl', 'data.table', 'DBI', 'digest', 'dplyr', 'fansi', 'foreach',
  'forecast', 'fracdiff', 'ggplot2', 'gh', 'haven', 'hexbin', 'hms',
  'htmltools', 'htmlwidgets', 'httpuv', 'IRkernel', 'jsonlite', 'KernSmooth',
  'knitr', 'later', 'lava', 'MASS', 'Matrix', 'mgcv', 'mime', 'ModelMetrics',
  'nlme', 'nycflights13', 'pillar', 'pkgbuild', 'pkgconfig', 'plyr',
  'prettyunits', 'processx', 'prodlim', 'promises', 'ps', 'purrr', 'quadprog',
  'R6', 'Rcpp', 'RcppArmadillo', 'RCurl', 'recipes', 'repr', 'rlang',
  'rmarkdown', 'roxygen2', 'RSQLite', 'rstudioapi', 'rvest', 'scales',
  'selectr', 'shiny', 'sparkl

In [14]:
BiocManager::install("devtools")
library("devtools")

Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'devtools'
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'backports', 'BH', 'bit', 'blob', 'broom', 'callr', 'caret',
  'cli', 'curl', 'data.table', 'DBI', 'digest', 'dplyr', 'fansi', 'foreach',
  'forecast', 'fracdiff', 'ggplot2', 'gh', 'haven', 'hexbin', 'hms',
  'htmltools', 'htmlwidgets', 'httpuv', 'IRkernel', 'jsonlite', 'KernSmooth',
  'knitr', 'later', 'lava', 'MASS', 'Matrix', 'mgcv', 'mime', 'ModelMetrics',
  'nlme', 'nycflights13', 'pillar', 'pkgbuild', 'pkgconfig', 'plyr',
  'prettyunits', 'processx', 'prodlim', 'promises', 'ps', 'purrr', 'quadprog',
  'R6', 'Rcpp', 'RcppArmadillo', 'RCurl', 'recipes', 'repr', 'rlang',
  'rmarkdown', 'roxygen2', 'RSQLite', 'rstudioapi', 'rvest', 'scales',
  'selectr', 'shiny', 'sparklyr', 'SQUAREM', 'stringi', 'survival', 'sys',
  'testthat', 'tidyr', 'tidyselect', 'tidyverse', 'tinytex', 'TTR', 'vctrs',
 

Begin here if you have already run this and created the data/gtex.rds file

You may need to adjust your working directory -- the data subdirectory is relative to the lifebitCloudOSDRE working directory

In [27]:
#obj <- downloadGTExV8(type='genes',file='data/gtex.rds')

Parsed with column specification:
cols(
  .default = col_double(),
  SAMPID = col_character(),
  SMCENTER = col_character(),
  SMPTHNTS = col_character(),
  SMTS = col_character(),
  SMTSD = col_character(),
  SMUBRID = col_character(),
  SMNABTCH = col_character(),
  SMNABTCHT = col_character(),
  SMNABTCHD = col_character(),
  SMGEBTCH = col_character(),
  SMGEBTCHD = col_character(),
  SMGEBTCHT = col_character(),
  SMAFRZE = col_character(),
  SMGTC = col_logical(),
  SMNUMGPS = col_logical(),
  SM550NRM = col_logical(),
  SM350NRM = col_logical(),
  SMMNCPB = col_logical(),
  SMMNCV = col_logical(),
  SMCGLGTH = col_logical()
  # ... with 2 more columns
)
See spec(...) for full column specifications.
Warning message:
“379 parsing failures.
  row   col           expected            actual                                       file
14837 SMGTC 1/0/T/F/TRUE/FALSE 6293764057_R01C01 '/tmp/Rtmp0DTXRW/phenodat51286d122c0b.txt'
14838 SMGTC 1/0/T/F/TRUE/FALSE 7983311048_R05C02 '/tmp/Rtmp0D

ERROR: Error in colnames(counts): object 'counts' not found


In [21]:
setwd('/mnt/shared/ec2-user/session_data')

In [22]:
getwd()

[1] "/mnt/shared/ec2-user/session_data"

In [ ]:
install("yarn")
library(yarn)

edgeR        (NA        -> 3.26.8  ) [CRAN]
gplots       (NA        -> 3.0.1.2 ) [CRAN]
limma        (NA        -> 3.40.6  ) [CRAN]
matrixStats  (NA        -> 0.55.0  ) [CRAN]
preproces... (NA        -> 1.46.0  ) [CRAN]
quantro      (NA        -> 1.18.0  ) [CRAN]
RCurl        (1.95-4.12 -> 1.98-1.1) [CRAN]
digest       (0.6.20    -> 0.6.24  ) [CRAN]
locfit       (NA        -> 1.5-9.1 ) [CRAN]
Rcpp         (1.0.2     -> 1.0.3   ) [CRAN]
gtools       (NA        -> 3.8.1   ) [CRAN]
gdata        (NA        -> 2.18.0  ) [CRAN]
caTools      (NA        -> 1.18.0  ) [CRAN]
KernSmooth   (2.23-15   -> 2.23-16 ) [CRAN]
hms          (0.5.0     -> 0.5.3   ) [CRAN]
R6           (2.4.0     -> 2.4.1   ) [CRAN]
BH           (1.69.0-1  -> 1.72.0-3) [CRAN]
minfi        (NA        -> 1.30.0  ) [CRAN]
doParallel   (NA        -> 1.0.15  ) [CRAN]
foreach      (1.4.7     -> 1.4.8   ) [CRAN]
ggplot2      (3.2.0     -> 3.2.1   ) [CRAN]
DBI          (1.0.0     -> 1.1.0   ) [CRAN]
RSQLite      (2.1.2     -> 2.2.0

Skipping 6 packages ahead of CRAN: Biobase, biomaRt, BiocGenerics, AnnotationDbi, IRanges, S4Vectors
Installing 85 packages: edgeR, gplots, limma, matrixStats, preprocessCore, quantro, RCurl, digest, locfit, Rcpp, gtools, gdata, caTools, KernSmooth, hms, R6, BH, minfi, doParallel, foreach, ggplot2, DBI, RSQLite, blob, pkgconfig, bit, rlang, vctrs, prettyunits, stringi, curl, jsonlite, mime, sys, cli, fansi, pillar, GenomicRanges, SummarizedExperiment, Biostrings, bumphunter, GenomeInfoDb, beanplot, nor1mix, siggenes, illuminaio, DelayedMatrixStats, mclust, genefilter, reshape, quadprog, data.table, GEOquery, DelayedArray, HDF5Array, BiocParallel, futile.logger, snow, XVector, doRNG, GenomicFeatures, annotate, GenomeInfoDbData, dplyr, tidyr, rhdf5, Rhdf5lib, base64, plyr, multtest, scrime, rngtools, tidyselect, lambda.r, futile.options, rtracklayer, purrr, lifecycle, zlibbioc, formatR, Rsamtools, GenomicAlignments, Rhtslib, scales, farver


This uploaded object is available by long names -- which might be nice to simplify ....

In [17]:
obj<-readRDS('/mnt/shared/ec2-user/session_data/mounted-data/lifebit-user-data-84f09c4a-623d-47db-b432-2880ba594b3b/deploit/teams/5db1ce1274081500dffba7b1/users/5c4cced085814700a85f8c7f/dataset/5e3ab72ee3474100f4708b3b/gtex.rds')

In [18]:
obj

ExpressionSet (storageMode: lockedEnvironment)
assayData: 55878 features, 17382 samples 
  element names: exprs 
protocolData: none
phenoData
  rowNames: GTEX-1117F-0226-SM-5GZZ7 GTEX-1117F-0426-SM-5EGHI ...
    GTEX-YEC3-1426-101806-SM-5PNXX (17384 total)
  varLabels: SAMPID SMATSSCR ... DTHHRDY (67 total)
  varMetadata: labelDescription
featureData
  featureNames: ENSG00000223972.5 ENSG00000227232.5 ...
    ENSG00000210196.2 (55878 total)
  fvarLabels: ensembl_gene_id hgnc_symbol ... gene_biotype (6 total)
  fvarMetadata: labelDescription
experimentData: use 'experimentData(object)'
Annotation:  

In [20]:
dim(phenoData(obj))

ERROR: Error in phenoData(obj): could not find function "phenoData"


In [ ]:
dim(obj)

In [ ]:
sample_names=as.vector(as.character(colnames(exprs(obj))))
head(sample_names)
length(sample_names)

In [ ]:
pheno_sample_names=as.vector(as.character(rownames(pData(obj))))
head(pheno_sample_names)
length(pheno_sample_names)

Okay - for some reason our phenotype data is larger than our expression data - I've written Joe Paulson about that.
In the meantime, make sure that the two sets are aligned.

In [ ]:
logical_match_names=pheno_sample_names %in% sample_names
length(logical_match_names)

In [ ]:
table(logical_match_names)


In [ ]:
pData(obj) <- (pData(obj)[logical_match_names==TRUE,])

Now we want to replace all *dashes* with _underscores_

In [ ]:
newSampID <- gsub('-','\\.',pData(obj)$SAMPID)

In [ ]:
head (newSampID)

In [ ]:
pData(obj)$SAMPID <- newSampID

In [ ]:
tissueFactors <- factor(tissues)

In [ ]:
table(tissueFactors)

In [ ]:
# SEX is coded 1 == Male
#              2 == Female
sex <- pData(obj)$SEX
age <- pData(obj)$AGE
#cod <- cause of death
cod <- pData(obj)$DTHHRDY
    

In [ ]:
table(sex)
table(age)
table(cod)

Now let us do the differential analysis - using EdgeR

In [15]:
BiocManager::install("edgeR")
library(edgeR)

Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'edgeR'
also installing the dependency ‘locfit’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'backports', 'BH', 'bit', 'blob', 'broom', 'callr', 'caret',
  'cli', 'curl', 'data.table', 'DBI', 'devtools', 'digest', 'dplyr',
  'ellipsis', 'fansi', 'foreach', 'forecast', 'fracdiff', 'ggplot2', 'gh',
  'haven', 'hexbin', 'hms', 'htmltools', 'htmlwidgets', 'httpuv', 'IRkernel',
  'jsonlite', 'KernSmooth', 'knitr', 'later', 'lava', 'MASS', 'Matrix', 'mgcv',
  'mime', 'ModelMetrics', 'nlme', 'nycflights13', 'pillar', 'pkgbuild',
  'pkgconfig', 'plyr', 'prettyunits', 'processx', 'prodlim', 'promises',
  'purrr', 'quadprog', 'R6', 'Rcpp', 'RcppArmadillo', 'RCurl', 'recipes',
  'repr', 'rlang', 'rmarkdown', 'roxygen2', 'RSQLite', 'rstudioapi', 'rvest',
  'scales', 'selectr', 'shiny', 'sparklyr', 'SQUAREM', 'stringi', 'survival',
  'sys', 'testthat', 'tidyr',

In [ ]:
x <- exprs(obj)

In [ ]:
dim(x)

To use the DGEList function from EdgeR, we need to transpose our x so that the length of group is equal
to the number of columns in our counts (x).

You will get an error in DGEList (counts = x, group = group) if the length of group is not equal to the number of columns in counts

In [ ]:
group <- factor(pData(obj)$SEX)

In [ ]:
y <- DGEList(counts=x, group=group)

I keep running out of memory on this step - so on my laptop after calculating the DGEList
I saved it and now I uploaded it to this larger memory machine

In [10]:
setwd("../../mounted-data/lifebit-user-data-e0354335-813e-4085-9693-9457f8507ff1/dataset/5e35bf40e3474100f467262b/")

In [11]:
getwd()

[1] "/mnt/shared/ec2-user/session_data/mounted-data/lifebit-user-data-e0354335-813e-4085-9693-9457f8507ff1/dataset/5e35bf40e3474100f467262b"

In [12]:
y <- readRDS("DGEy.rds")

In [13]:
attributes(y)

$class
[1] "DGEList"
attr(,"package")
[1] "edgeR"

$names
[1] "counts"  "samples"

In [16]:
y <- calcNormFactors(y)

In [17]:
saveRDS(y, file = "DGENormFactorsy.rds")

We only want to keep those events that are greater than the first quartile (25%),
this is done using all non-zero (>1) events >= 0.25 min(table(pData(obj)$SEX))

Recall SEX is coded 1 for male, 2 for female

In [ ]:
groups <- pData(obj)$SEX
keep.events <- rep(TRUE, nrow(y))
for (group in c(1,2)) {
    keep.events <- keep.events & 
                   rowSums(cpm(y[,groups %in% group]) > 1) >= 0.25*min(table(groups))
}
